In [ ]:
from binary_models.load_se_rt import load_community_model, model_folder

In [ ]:
import pandas as pd
import numpy as np
import cobra

from micom import Community
from micom.workflows import  grow, tradeoff
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

In [ ]:
# first time running, add prep_files=True
# or if there are any model updates, or if you aren't sure but want to be
community, manifest = load_community_model(prep_files=False)

In [ ]:
# Set variable to become new medium
grow_medium_to_change = community.medium
#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
grow_medium_to_change["EX_sucr_m"] = 1
#grow_medium_to_change["EX_leu__L_m"] = 1

In [ ]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

In [ ]:
result_grow = grow(
    manifest,
    model_folder=model_folder,
    medium=grow_medium,
    tradeoff=0.01,
    threads=2,
    presolve=True
)

In [ ]:
result_grow.growth_rates

In [ ]:
result_grow.exchanges

In [ ]:
exchanges = result_grow.exchanges.copy()

In [ ]:
exchanges = exchanges[['taxon', 'reaction', 'flux', 'metabolite', 'direction']]

In [ ]:
exchanges.to_csv("se_rt_exchanges.csv", index=False)

In [ ]:
grow_plot = plot_growth(result_grow)

In [ ]:
grow_plot.view()

In [ ]:
trades = tradeoff(
    manifest,
    model_folder,
    grow_medium,
    tradeoffs=np.arange(0.1, 1.0 + 1e-6, 0.1),
    threads=8,
    atol=None,
    rtol=None,
    presolve=False,
)

In [ ]:
tr_plot = plot_tradeoff(trades)

In [ ]:
tr_plot.view()